# Rag from Scratch
- Step-1 : Extract text
- Step-2 : Chunk the extracted text
- Step-3 : Create a vector store with the chunks
- Step-4 : Create a retriever which returns the relevant chunks
- Step-5 : Build context from the relevant chunk texts
- Step-6 : Build the RAG pipeline
- Step-7 : Run the RAG pipeline to get the answer.

In [ ]:
from dotenv import load_dotenv
import os

# Read .env file
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if api_key:
    print("OpenAI API key loaded successfully!")
else:
    print("Error: OPENAI_API_KEY not found")

OpenAI API key loaded successfully!


### Extract Text

In [2]:
from typing import List
from PyPDF2 import PdfReader

def text_extract(pdf_path: str) -> str:
    """
    Extracts text from all pages of a given PDF file.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        str: Extracted text from the PDF, concatenated with newline separators.
    """

    # An empty list to store extracted text from PDF pages
    pdf_pages = []

    # Open the PDF file in binary read mode
    with open(pdf_path, 'rb') as file:

        # Create a PdfReader object to read the PDF
        pdf_reader = PdfReader(file)

        # Iterate through all pages in the PDF
        for page in pdf_reader.pages:

            # Extract text from the current page
            text = page.extract_text()

            # Append the extracted text to the list
            pdf_pages.append(text)

    # Join all extracted text using newline separator
    pdf_text = "\n".join(pdf_pages)

    # Return the extracted text as a single string
    return pdf_text

In [ ]:
import requests

# 1. Get project's root directory
current_dir = os.getcwd()

# 2. Define a path for the 'content' folder
content_dir = os.path.join(current_dir, "content")

# 3. Create the 'content' folder if it doesn't already exist
os.makedirs(content_dir, exist_ok=True)

# 4. Define the full, correct path for the PDF
pdf_path = os.path.join(content_dir, "sparql_query_translation.pdf")

# Download the PDF file
pdf_url = 'https://arxiv.org/pdf/2507.10045.pdf'
response = requests.get(pdf_url)

with open(pdf_path, 'wb') as file:
    file.write(response.content)

# Extract the text
pdf_text = text_extract(pdf_path)
print(pdf_text[:300])

July 2025
Automating SPARQL Query Translations
between DBpedia and Wikidata
Malte Christian BARTELSa,1, Debayan BANERJEEaand Ricardo USBECKa
aLeuphana University of Lüneburg, Lüneburg, Germany
ORCiD ID: Malte Christian Bartels https://orcid.org/0009-0006-2113-3322, Debayan
Banerjee https://orcid.org


### Chunk Text

In [4]:
from typing import List
import re
from collections import deque


def text_chunk(text: str, max_length: int = 1000) -> List[str]:
    """
    Splits a given text into chunks while ensuring that sentences remain intact.

    The function maintains sentence boundaries by splitting based on punctuation
    (. ! ?) and attempts to fit as many sentences as possible within `max_length`
    per chunk.

    Args:
        text (str): The input text to be chunked.
        max_length (int, optional): Maximum length of each chunk. Default is 1000.

    Returns:
        List[str]: A list of text chunks, each containing full sentences.
    """

    # Split text into sentences while ensuring punctuation (. ! ?) stays at the end
    sentences = deque(re.split(r'(?<=[.!?])\s+', text.replace('\n', ' ')))

    # An empty list to store the final chunks
    chunks = []

    # Temporary string to hold the current chunk
    chunk_text = ""

    while sentences:
        # Access sentence from the deque and strip any extra spaces
        sentence = sentences.popleft().strip()

        # Check if the sentence is non-empty before processing
        if sentence:
            # If adding this sentence exceeds max_length and chunk_text is not empty, store the current chunk
            if len(chunk_text) + len(sentence) > max_length and chunk_text:

                # Save the current chunk
                chunks.append(chunk_text)

                # Start a new chunk with the current sentence
                chunk_text = sentence
            else:
                # Append the sentence to the current chunk with a space
                chunk_text += " " + sentence

    # Add the last chunk if there's any remaining text
    if chunk_text:
        chunks.append(chunk_text)

    return chunks

In [5]:
chunks = text_chunk(pdf_text)
print(f"Number of chunks ={len(chunks)}")
print(chunks[0])

Number of chunks =59
 July 2025 Automating SPARQL Query Translations between DBpedia and Wikidata Malte Christian BARTELSa,1, Debayan BANERJEEaand Ricardo USBECKa aLeuphana University of Lüneburg, Lüneburg, Germany ORCiD ID: Malte Christian Bartels https://orcid.org/0009-0006-2113-3322, Debayan Banerjee https://orcid.org/0000-0001-7626-8888, Ricardo Usbeck https://orcid.org/0000-0002-0191-7211 Abstract. Purpose : This paper investigates whether state-of-the-art Large Lan- guage Models (LLMs) can automatically translate SPARQL between popular Knowledge Graph (KG) schemas. We focus on translations between the DBpedia and Wikidata KG, and later on DBLP and OpenAlex KG. This study addresses a notable gap in KG interoperability research by evaluating LLM performance on SPARQL-to-SPARQL translation.


### Create the Vector Store

In [6]:
# Set up Chromadb
import chromadb
from chromadb.utils import embedding_functions
from chromadb.api.models import Collection

def create_vector_store(db_path: str) -> Collection:
    """
    Creates a persistent ChromaDB vector store with OpenAI embeddings.

    Args:
        db_path (str): Path where the ChromaDB database will be stored.

    Returns:
        Collection: A ChromaDB collection object for storing and retrieving embedded vectors.
    """

    # Initialize a ChromaDB PersistentClient with the specified database path
    client = chromadb.PersistentClient(path=db_path)

    # Create an embedding function using OpenAI's text embedding model
    embeddings = embedding_functions.OpenAIEmbeddingFunction(
        api_key=api_key,  # Retrieve API key from user data
        model_name="text-embedding-3-small"  # Specify the embedding model
    )

    # Create a new collection in the ChromaDB database with the embedding function
    db = client.create_collection(
        name="pdf_chunks",  # Name of the collection where embeddings will be stored
        embedding_function=embeddings  # Apply the embedding function
    )

    # Return the created ChromaDB collection
    return db

In [ ]:
# Insert chunks into vector store
import uuid

def insert_chunks_vectordb(chunks: List[str], db: Collection, file_path: str) -> None:
    """
    Inserts text chunks into a ChromaDB vector store with metadata.

    Args:
        chunks (List[str]): List of text chunks to be stored.
        db (Collection): The ChromaDB collection where the chunks will be inserted.
        file_path (str): Path of the source file for metadata.

    Returns:
        None
    """

    # Extract the file name from the given file path
    file_name = os.path.basename(file_path)

    # Generate unique IDs for each chunk
    id_list = [str(uuid.uuid4()) for _ in range(len(chunks))]

    # Create metadata for each chunk, storing the chunk index and source file name
    metadata_list = [{"chunk": i, "source": file_name} for i in range(len(chunks))]

    # Define batch size for inserting chunks to optimize performance
    batch_size = 40

    # Insert chunks into the database in batches
    for i in range(0, len(chunks), batch_size):
        end_id = min(i + batch_size, len(chunks))  # Ensure we don't exceed list length

        # Add the batch of chunks to the vector store
        db.add(
            documents=chunks[i:end_id],
            metadatas=metadata_list[i:end_id],
            ids=id_list[i:end_id]
        )

    print(f"{len(chunks)} chunks added to the vector store")

### Retrieve Chunks

In [8]:
from typing import Any, List

def retrieve_chunks(db: Collection, query: str, n_results: int = 2) -> List[Any]:
    """
    Retrieves relevant chunks from the  vector store for the given query.

    Args:
        db (Collection): The vector store object
        query (str): The search query text.
        n_results (int, optional): The number of relevant chunks to retrieve. Defaults to 2.

    Returns:
        List[Any]: A list of relevant chunks retrieved from the vector store.
    """

    # Perform a query on the database to get the most relevant chunks
    relevant_chunks = db.query(query_texts=[query], n_results=n_results)

    # Return the retrieved relevant chunks
    return relevant_chunks

### Build Context

In [9]:
def build_context(relevant_chunks) -> str:
    """
    Builds a single context string by combining texts from relevant chunks.

    Args:
        relevant_chunks: relevant chunks retrieved from the vector store.

    Returns:
        str: A single string containing all document chunks combined with newline separators.
    """

    # combine the text from relevant chunks with newline separator
    context = "\n".join(relevant_chunks['documents'][0])

    # Return the combined context string
    return context


### Build RAG Pipeline

In [10]:
import os
from typing import Tuple

def get_context(pdf_path: str, query: str, db_path: str) -> Tuple[str, str]:
    """
    Retrieves the relevant chunks from the vector store and then builds context from them.

    Args:
        pdf_path (str): The file path to the PDF document.
        query (str): The query string to search within the vector store.
        db_path (str): The file path to the persistent vector store database.

    Returns:
        Tuple[str, str]: A tuple containing the context related to the query and the original query string.
    """

    # Check if the vector store already exists
    if os.path.exists(db_path):
        print("Loading existing vector store...")

        # Initialize the persistent client for the existing database
        client = chromadb.PersistentClient(path=db_path)

        # Create the embedding function using OpenAI embeddings
        embeddings = embedding_functions.OpenAIEmbeddingFunction(
            api_key=api_key,  # Fetch API key
            model_name="text-embedding-3-small"      # Specify the embedding model
        )

        # Get the collection of PDF chunks from the existing vector store
        db = client.get_collection(name="pdf_chunks", embedding_function=embeddings)
    else:
        print("Creating new vector store...")

        # Extract text from the provided PDF
        pdf_text = text_extract(pdf_path)

        # Chunk the extracted text
        chunks = text_chunk(pdf_text)

        # Create a new vector store
        db = create_vector_store(db_path)

        # Insert the text chunks into the vector store
        insert_chunks_vectordb(chunks, db, pdf_path)

    # Retrieve the relevant chunks based on the query
    relevant_chunks = retrieve_chunks(db, query)

    # Build the context from the relevant chunks
    context = build_context(relevant_chunks)

    # Return the context and the original query
    return context, query

In [11]:
def get_prompt(context: str, query: str) -> str:
    """
    Generates a rag prompt based on the given context and query.

    Args:
        context (str): The context the LLM should use to answer the question.
        query (str): The user query that needs to be answered based on the context.

    Returns:
        str: The generated rag prompt.
    """

    # Format the prompt with the provided context and query
    rag_prompt = f""" You are an AI model trained for question answering. You should answer the
    given question based on the given context only.
    Question : {query}
    \n
    Context : {context}
    \n
    If the answer is not present in the given context, respond as: The answer to this question is not available
    in the provided content.
    """

    # Return the formatted prompt
    return rag_prompt

In [12]:
from litellm import completion

def get_response(rag_prompt: str) -> str:
    """
    Sends a prompt to the OpenAI LLM and returns the answer.

    Args:
        rag_prompt (str): The rag prompt.

    Returns:
        str: The LLM generated answer.
    """
    # Specify the LLM to use
    model = "openai/gpt-4o-mini"

    # Prepare the message to be sent to the model
    messages = [{"role": "user", "content": rag_prompt}]

    # Call the completion function to get a response from the model
    response = completion(model=model, messages=messages, temperature=0)

    # Return the answer
    answer = response.choices[0].message.content
    return answer

In [13]:
def rag_pipeline(pdf_path: str, query: str, db_path: str) -> str:
    """
    Runs a Retrieval-Augmented Generation (RAG) pipeline to retrieve context from a vector store,
    generate the rag prompt, and then get the answer from the model.

    Args:
        pdf_path (str): The file path to the PDF document from which context is extracted.
        query (str): The query for which a response is needed, based on the context.
        db_path (str): The file path to the persistent vector store database used for context retrieval.

    Returns:
        str: The model's response based on the context and the provided query.
    """

    # get the context
    context, query = get_context(pdf_path, query, db_path)

    # Generate the rag prompt based on the context and query
    rag_prompt = get_prompt(context, query)

    # Get the response from the model using the rag prompt
    response = get_response(rag_prompt)

    # Return the model's response
    return response


### Run RAG Pipeline

In [ ]:
# 1. Get your *actual* current directory (e.g., /path/to/rag_learning)
current_dir = os.getcwd()

# 2. Build the database path relative to your current directory
# This will create a path like: /path/to/rag_learning/db/chroma_db_pdf
persistent_directory = os.path.join(current_dir, "db", "chroma_db_pdf")

# 3. Build the PDF path relative to your current directory
# This will create a path like: /path/to/rag_learning/content/sparql_query_translation.pdf
pdf_path = os.path.join(current_dir, "content", "sparql_query_translation.pdf")

# 4. The query is fine
query = "What is the purpose of this paper?"

Project Directory: c:\Users\malte\Desktop\Coding\rag_learning
Database Path:     c:\Users\malte\Desktop\Coding\rag_learning\db\chroma_db_pdf
PDF Path:          c:\Users\malte\Desktop\Coding\rag_learning\content\sparql_query_translation.pdf


In [15]:
# Run the RAG pipeline
answer = rag_pipeline(pdf_path, query, persistent_directory)

Creating new vector store...
59 chunks added to the vector store


In [16]:
print(f"Query:{query}")
print(f"Generated answer:{answer}")

Query:What is the purpose of this paper?
Generated answer:The answer to this question is not available in the provided content.


In [19]:
# RAG query
query = "How was Zero-Shot prompting used in the paper?"

# Run the RAG pipeline
answer = rag_pipeline(pdf_path, query, persistent_directory)

print(f"Query:{query}")
print(f"Generated answer:{answer}")

Loading existing vector store...
Query:How was Zero-Shot prompting used in the paper?
Generated answer:Zero-Shot prompting was used in the paper as a baseline approach to assess the LLMs' inherent ability to translate SPARQL queries without any task-specific examples and without the explicit entity-relation (ER) mapping. The prompt included only the natural language query (NLQ), source query, knowledge graph (KG) names, and output instruction. This setup aimed to highlight the challenges LLMs face when relying solely on their pre-trained knowledge. Additionally, a variant of Zero-Shot prompting that included ER mapping was used to address limitations related to ambiguity in mapping schema elements.
